In [ ]:
# 10/03 Q test. bug. emailed author.
#     is Q having a unit of 1/V? give an 1/1000 to Q

In [22]:
from brian2 import *
from numpy import matlib
import numpy as np

In [23]:
start_scope()

defaultclock.dt = 0.025*ms
duration = 1*second



# INPUT SEGMENT

n = 1
tau = 1*ms
eqs = '''
dv/dt = (v0 - v) / tau : volt (unless refractory)
v0 : volt
'''
input_lif = NeuronGroup(n, eqs, threshold='v > 50*mV', reset='v = -60*mV',
                    refractory=95*ms, method='linear')
input_lif.v = 0*mV
input_lif.v0 = 60*mV

# monitor = SpikeMonitor(input_lif)
mon_input = StateMonitor(input_lif,'v',record = True)

# # Two-compartment structure
# morpho = Soma(30*um)
# morpho.spine = Cylinder(diameter=1*um, length=300*um, n=1)



In [ ]:
# FUNCTION SECTION
zetap = 30.  # This is 40 in Poirazi paper
tempc = 23.*kelvin

ca = 2.*umolar # ca_out

xx = 0.0853*(273.16 + tempc/kelvin)/2  # kelvin
tadj = 2.3**((tempc/kelvin-23)/10)   
# unitless
q = 96480/(8.315*(273.16+tempc/kelvin))  
qt = 5^((tempc/kelvin-24)/10)  

qma = 0.000480*(1/ms)
qmb = 0.280000*(1/ms)
qhat = 1.


@check_units(v=mV, result=1)
def zeta(v):
    return -1.5-(1/(1+exp((v+zetap)/5))) 

@check_units(v=mV, result=1)
def zeta2(v):
    return -1.8-(1/(1+exp((v/mV+40)/5)))  




@check_units(z=1, result=1)  # z is volt, so when used should do eff(z/mV). see ghk(volt)
def eff(z)
    return (1-z/2)*eff2(z)+(z/(exp(z)-1))*eff3(z)  

@check_units(z=1, result=1)  
def eff2(z)
    return heaviside(0.0001-abs(z), 1)  

@check_units(z=1, result=1)  
def eff3(z)
    return heaviside(abs(z)-0.0001, 1)  


@check_units(v=mV, chi = umolar, result=umolar)
def ghk(v,chi)
    return -xx*(1-((chi/ca)*exp(v/xx)))*eff(v/mV/xx)  

@check_units(v=mV, result=1)
def salfa(v)
    return -0.055*(v/mV+27.01)/(exp((-v/mV-27.01)/3.8)-1)  

@check_units(v=mV, result=1)
def sbeta(v) 
    return 0.94*exp((-v/mV-63.01)/17) 


t_st = 10*defaultclock.dt
impulse_duration = 50*defaultclock.dt*ms  # 1.25*ms
impulse_str = 62.5*uamp
period = 50*ms

@check_units(t=ms, result=1)
def frost(t) 
    return (np.heaviside((t-t_st)/ms,0)*np.heaviside(sin(2*pi*(t-t_st-impulse_duration)/period))
            *np.heaviside(sin(2*pi*(t-t_st-2*impulse_duration)/period))

In [3]:


Ne = 1

# some calcium-related vars, table 2
pHC = 4.0*umolar
aHC = 0.6*umolar
pHN = 4.
aHN = 3.
alphav = 1.
alphab = 5.
alphad = 1.
thetav = 2.*umolar
thetad = 2.6
thetab = 0.55
Cp = 5.
Cd = 4.
taup = 500*ms
taua = 5*ms
tauv = 10*ms
taud = 250*ms
taub = 40*ms
sigmav = -0.05*umolar
sigmad = -0.01
sigmab = -0.02
p = 0.3
d = 0.01
kd = -0.002
kp = -0.1

# some calcium-related vars, concentration
phi = 0.01*umolar/ms/(uamp*cm**-2)
beta = 0.083/ms
chi0 = 0.05*umolar
nbuff = 6.*umolar



# E cells stats
Cme = 1*ufarad/cm**2
Ele = -67*mV
ENae = 50*mV
Ek = -100*mV
gle = 0.1*msiemens/cm**2
gKe = 80*msiemens/cm**2

# AMPA E-E
syn_mod = 1.

gampae = 0.05*syn_mod*msiemens/cm**2

# gampae = 0.00*syn_mod/(Ne-1)*msiemens/cm**2

#ggabae = 0.03*syn_mod/(Ni)*msiemens/cm**2


# NMDA (excitatory)
g_syn_nmda = 0.3*syn_mod*msiemens/cm**2
g_cal_nmda_0 = 25*syn_mod*msiemens/cm**2
# g_NMDA_I = 0.258 * nS * 800. / N_E
V_cal = 140*mV
tausnmdar = 1*ms
tausnmdad = 100*ms
# Mg2 = 1.
Mg2 = 2.


# Ina params
gNae = 30*msiemens/cm**2
naatt = 1.
taumna = 0.05*ms
# tauhna = 0.5*ms   # dendrite
tauhna = 1.*ms  # soma



# Ik params
gkdr = 14*msiemens/cm**2
taukdr = 3.5*ms

# Ia
ga = 7.5*msiemens/cm**2

# Imahp
gmahp = 25*msiemens/cm**2


# TEST
# Isyn = 0*amp*meter**-2
# Ileak = 0*amp*meter**-2
# I_cal_nmda = 0*amp*meter**-2
# Icoup = 0*amp*meter**-2
# gNae = 0*msiemens/cm**2
# # ga = 0*msiemens/cm**2
# gmahp = 0*msiemens/cm**2
# gkdr = 0*msiemens/cm**2


eqs_soma = '''
dv/dt = ( ( Ileak +  Ina + Ik + Ica + Icoup + Iin))/Cme : volt

Ileak = gl * (El-v) : amp*meter**-2


Ina = gNa * mna**2 * hna * (ENa-v) : amp*meter**-2
# soma activation
alphams = 0.32*(-46.9-v/mV)/(exp((-46.9-v/mV)/4.0)-1.0)/ms :Hz
betams = 0.28*(v/mV+19.9)/(exp((v/mV+19.9)/5.0)-1.0)/ms :Hz
mna = alphams/(alphams+betams) : 1
dh_soma/dt=alphahs-(alphahs+betahs)*h_soma  :1
alphahs = 0.128*exp((-43.0-v/mV)/18.0)/ms :Hz
betahs = 4.0/(1.0+exp((-20.0-v/mV)/5.0))/ms :Hz


Ik = Ia + Ikdr + Imahp : amp*meter**-2

Ikdr = gkdr * n_soma * (Ek - v) : amp*meter**-2
dn_soma/dt = alphans-(alphans+betans)*n_soma : 1 
alphans =  0.016*(-24.9-v/mV)/(exp((-24.9-v/mV)/5.0)-1.0)/ms : Hz  
betans =   0.25*exp(-1.0-0.025*v/mV)/ms  : Hz

Ia = ga * a_soma * b_soma * (Ek-v) : amp*meter**-2
da_soma/dt=(a_somainf-a_soma)/(a_somatau)  : 1
a_somainf=1/(1+a_smaalf)  : 1
a_smaalf=exp(0.001*zeta(v)*(v/mV-11)*q)  : 1
a_smabet=exp(0.00055*q*(v/mV-11)*zeta(v)) : 1
a_somatau=clip(a_smabet/((1+a_smaalf)*qt*0.05),0.1, inf)*ms : msecond

Imahp = gmahp * qm_soma * (Ek - v) : amp*meter**-2
dqm_soma/dt = (qm_smainf-qm_soma)/(qm_smatau)  :1
qm_smainf = qhat*qm_smaalf*qm_smatau  : 1
qm_smaalf = qma*chisoma/umolar/(0.001*chisoma/umolar + 0.18*exp(-1.68*v/mV*q)) : Hz 
qm_smabet = (qmb*exp(-0.022*v/mV*q))/(exp(-0.022*v/mV*q)+0.001*chisoma/umolar) : Hz  
qm_smatau = 1/(qm_smaalf+qm_smabet)  : ms


Ica = -gcalsoma*s_soma*ghk(v,chisoma)*(1/(1+chisoma/umolar))  
ds_soma/dt=(s_somainf-s_soma)/(s_somatau)  : 1
s_somainf=salfa(v)/(salfa(v)+sbeta(v))  : 1
s_somatau=1/(5*(salfa(v)+sbeta(v)))*ms  : ms


Icoup : amp*meter**-2


#Iin : amp*meter**-2 # specify later
Iin = impulse_str*frost(t)*cm**2 : amp*meter**-2 # specify later



dchi/dt = phi_soma*I_cal_nmda - beta*(chi-chi0) -(beta/nbuff)*chi**2 : mmolar
dP/dt = ((10*(chi/pHC)**pHN / (1+(chi/pHC)**pHN))-Cp*A*P)/taup : 1
dA/dt = (((chi/aHC)**aHN/(1+(chi/aHC)**aHN))-A)/taua : 1
dV/dt = ( alphav/(1+exp((chi-thetav)/sigmav)) -V)/tauv : 1
dB/dt = ( alphab/(1+exp((A-thetab)/sigmab)) -B -Cd*V*B)/taub : 1
dD/dt = ( alphad/(1+exp((B-thetad)/sigmad)) -D)/taud : 1
'''

# backup
# for qm in Imahp
# dqm_soma/dt = (qm_smainf-qm_soma)/(qm_smatau) :1
# qm_smainf = qhat*qm_smaalf*qm_smatau  
# qm_smaalf = qma*chisoma/(0.001*chisoma+0.18*exp(-1.68*v*q))  
# qm_smabet = (qmb*exp(-0.022*v*q))/(exp(-0.022*v*q)+0.001*chisoma)  
# qm_smatau = 1/(qm_smaalf+qm_smabet)  

Ecells = NeuronGroup(Ne, eqse, threshold='v>0*mV',refractory='v > 0*mV',method='rk4')

# Me = StateMonitor(Ecells, 'v', record=True)



# ---------------- Initialization ---------------- #
Ecells.P = 0.3
Ecells.A = 0.5
Ecells.V = 0.01
Ecells.B = 2.5
Ecells.D = 0.01



Ecells.v = -60*mV
Ecells.chi = 0.5*umolar

g_cal_nmda = g_cal_nmda_0


# # ---------------- Synapses ---------------- #

# Coupling
# eqs_coup = '''
# Ileak = 1.125*(v_post - v_pre)/mV*uA
# '''

# C_coup = Synapses(Ecells, Ecells, model=eqs_coup, on_post = eqs_nmda_post,
#                  method='rk4')
# C_coup.connect(condition='i != j')


# INPUT FROM LIF
g_input = 0.1*msiemens/cm**2
input_eqs = '''
            I_input = 1/Cme * g_input * (v_post - v_pre)
'''

input_syn = Synapses(input_lif, Ecells, model=input_eqs, method='rk4')
input_syn
# C_coup.connect(condition='i != j')


# mon = StateMonitor(Ecells, ['v','Ik','Ina'],record = True)
# mon = StateMonitor(Ecells, ['v','ksi','Ia','alphama','betama','alphaha','betaha'],record = True)
# mon = StateMonitor(Ecells, ['v','Ina','Ik','Ia','Ikdr','Imahp'],record = True)



# mon = StateMonitor(Ecells, ['Ina','mna','hna','sna','tausna','snainf'],record = True)


# mon = StateMonitor(Ecells, ['v','Ik','Ina','ma','ha','alphama','betama'],record = True)


# mon_E = StateMonitor(Ecells, ['v','chi','P','V','A','D','B','I_NMDA_rec','I_cal_nmda','I_syn_nmda'],record = [0,1])
# mon_sp = SpikeMonitor(Ecells,record = True)



SyntaxError: invalid syntax (<unknown>, line 1)

In [ ]:
run(duration,report='stdout')

In [49]:
# (mon.v[0][0:200])
(mon.v.shape)

(1, 120)

In [53]:
# plot(mon.t,mon.v[0][:])
#      -mon.Ia[0][:])
# plot(mon.v[0][0:150]/mV)
# plot(mon.t,mon.v[0][0:150]/mV)
# mon.Ia[0][0:100]

In [67]:
# -1.5*(1/mV) - 1/(1+exp((-mon.v[0][:]+40*mV)/(5*mV)))*(1/mV)
# plot(mon.v[0][:])
mon.v[0][0:4]

array([-60.        , -59.98622109, -43.55243546,          nan]) * mvolt

In [6]:
# exp(1e-3 * mon.ksi[0] * (mon.v[0]-11*mV) * Q)
Q

0.43198536 * volt ** -1

In [131]:
1e-3 * mon.ksi[0] * (mon.v[0]-11*mV) * Q

array([182.93585598,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan,          nan,          nan])

In [64]:
# Q = F/coulomb/(R/(joule/degC)*(T+degC)/kelvin)
Q


0.020244132317562454

In [2]:
import numpy as np
import numpy.matlib
a0 = np.array(1)
np.matlib.repmat(a0, 2, 3)


array([[1, 1, 1],
       [1, 1, 1]])

In [2]:
start_scope()

tau = 10*ms
eqs = '''
dv/dt = (0.5-v)/tau : 1
'''

G = NeuronGroup(1, eqs, threshold='v>0.8', reset='v = 0', method='exact')


S = Synapses(stimulation, neuron, on_pre='v += w')

S.connect(i=0, j=0)

NameError: name 'stimulation' is not defined

In [16]:
from brian2.units import *
@check_units(I=1, R=1, wibble=1, result=1)
def getvoltage(I, R, **k):
    return np.heaviside(I*R,1)
getvoltage(1,1)

1.0

In [20]:
test1 = np.array([1,2])
test2 = np.array([1,2])
test1*test2

array([1, 4])